# **Compliance GPT with LangChain**

## **Setup**

In [14]:
# import nest_asyncio
# nest_asyncio.apply()

import warnings
warnings.filterwarnings("ignore")

## **Config**

In [15]:
from utils.config import get_config

config = get_config()

In [16]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [17]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_4O_MINI, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Load Vector Store Index**

In [18]:
from database.vector_store.vector_store import RedisIndexManager, PineconeIndexManager, PostgresIndexManager, ElasticIndexManager

# es_bi = ElasticIndexManager(index_name='bi', embed_model=embed_model, config=config)
# vector_store_bi = es_bi.load_vector_index()

# es_ojk = ElasticIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = es_ojk.load_vector_index()

# es_ket = ElasticIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config)
# vector_store_ket = es_ket.load_vector_index()

# es_rek = ElasticIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_rek = es_rek.load_vector_index()

# postgres_bi = PostgresIndexManager(index_name='bi', embed_model=embed_model, config=config)
# vector_store_bi = postgres_bi.load_vector_index()

# postgres_ojk = PostgresIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = postgres_ojk.load_vector_index()

# postgres_ket = PostgresIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_ket = postgres_ket.load_vector_index()

# postgres_rek = PostgresIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
# vector_store_rek = postgres_rek.load_vector_index()

redis_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
vector_store_bi = redis_bi.load_vector_index()

redis_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
vector_store_ojk = redis_ojk.load_vector_index()

redis_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
vector_store_ket = redis_sikepo_ket.load_vector_index()

redis_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
vector_store_rek = redis_sikepo_rek.load_vector_index()

# pinecone_ojk = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = pinecone_ojk.load_vector_index()

# pinecone_bi = PineconeIndexManager(index_name='bi', embed_model=embed_model, config=config)
# vector_store_bi = pinecone_bi.load_vector_index()

# pinecone_ket = PineconeIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_ket = pinecone_ket.load_vector_index()

# pinecone_rek = PineconeIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
# vector_store_rek = pinecone_rek.load_vector_index()

Loaded index 'bi'.
Loaded index 'ojk'.
Loaded index 'sikepo-ketentuan-terkait'.
Loaded index 'sikepo-rekam-jejak'.


In [19]:
from database.vector_store.neo4j_graph_store import Neo4jGraphStore

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

## **Get Retriever**

In [20]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo

top_n = 10

retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_n=top_n,
                                  llm_model=llm_model, embed_model=embed_model, config=config)
retriever_bi = get_retriever_bi(vector_store=vector_store_bi, top_n=top_n,
                                llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, top_n=top_n,
                                   llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, top_n=top_n,
                                   llm_model=llm_model, embed_model=embed_model, config=config)

In [21]:
# retriever_bi.invoke("What is the maximum transaction of QRIS?")

## **Create Chain**

In [22]:
from database.chat_store import RedisChatStore, MongoDBChatStore, PostgresChatStore, ElasticChatStore

# chat_store = ElasticChatStore(config=config, k=3)
# chat_store = PostgresChatStore(config=config, k=3)
chat_store = RedisChatStore(k=3, config=config, db_id=1)
# chat_store = MongoDBChatStore(config=config, k=4)

In [23]:
from chain.rag_chain import create_chain_with_chat_history, create_sequential_chain
from chain.chain_sikepo.graph_cypher_sikepo_chain import graph_rag_chain


graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)

chain = create_sequential_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk,
    retriever_bi=retriever_sikepo_ket,
    retriever_sikepo_ketentuan=retriever_sikepo_ket,
    retriever_sikepo_rekam=retriever_sikepo_rek,
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **Chat Store Monitor**

In [24]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Apa itu Akad Tijarah?
Human: Jika saya ingin membuat peraturan terkait bank digital peraturan apa yang perlu saya perhatikan?


In [25]:
# to make the chat history empty
chat_store.clear_all()

#### **With Question, Answer, and Context**

In [26]:
# from chain.rag_chain import get_response

# response = get_response(
#     chain=chain_history,
#     question="Apa itu Akad Tijarah?",
#     user_id=USER_ID,
#     conversation_id=CONVERSATION_ID
# )

# response

In [27]:
# print(response['answer'])


In [28]:
# print(response['context'])

#### **Streaming**

In [29]:
from chain.rag_chain import print_answer_stream

question = "Jika saya ingin membuat peraturan terkait bank digital peraturan apa yang perlu saya perhatikan?"

print_answer_stream(chain=chain_history, question=question, user_id=USER_ID, conversation_id=CONVERSATION_ID)

Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Failed to write data to connection IPv4Address(('76b9a325.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Jika Anda ingin membuat peraturan terkait bank digital, Anda perlu memperhatikan beberapa peraturan yang relevan, antara lain:

1. **Peraturan Otoritas Jasa Keuangan (POJK) Nomor 21 Tahun 2023** tentang Layanan Digital oleh Bank Umum. Peraturan ini mengatur tentang pengembangan dan inovasi dalam memberikan layanan digital oleh bank, serta manajemen risiko, keamanan data nasabah, dan perlindungan konsumen. 

   Source: [21 Tahun 2023](https://www.ojk.go.id/id/regulasi/Documents/Pages/Layanan-Digital-oleh-Bank-Umum/POJK%2021%20Tahun%202023.%20Layanan%20Digital%20oleh%20Bank%20Umum.pdf) (halaman 1)

2. **Peraturan Otoritas Jasa Keuangan (POJK) Nomor 12/POJK.03/2018** tentang Penyelenggaraan Layanan Perbankan Digital oleh Bank Umum. Peraturan ini perlu diperhatikan karena akan digantikan oleh POJK 21 Tahun 2023, namun masih relevan untuk memahami ketentuan yang ada sebelumnya.

   Source: [12/POJK.03/2018](https://www.ojk.go.id/id/regulasi/Documents/Pages/Penyelenggaraan-Layanan-Perbankan-